In [1]:
import pandas as pd
import numpy as np


df_credit_reports = pd.read_csv('./dataset/credit_reports.csv')
df_users = pd.read_csv('./dataset/users.csv')


# Business Understanding

This phase focuses on understanding the project objective and requirements from a business perspective, and then converting this knowledge into a data mining problem definition, and a preliminary plan designed to achieve the objectives.

## Objectives:

- 1.- Pick the best clients you will give a loan to, based on the model you created. It could be as complex as you decide (even as simpler as knock out rules), as long as the metrics support it

- 2.- Propose an amount to be lended to those clients and a term in which the loan will need to be paid back.

- 3.- Finally choose an anual interest rate the lended amount must have in order to be profitable.

## Requirements based on objectives:

Without expertise in this subject, my approach to this problem 
is to minize loosses, as oppose to, for instance, maximize earnings.

I'll use a double lock for this purpose: 
- Minimize the number of misclassifying untrustworthy clients
- Estimate the losses would be produced by misclassification, given an amount lent and a term. This calculation, togueder with the calculation of the money paid back by trustworthy client, will fix the interest rate necessary to make the business profitable.

# Metodology:

### Strategy to reach solution 1  

Classification model to choose creditworthy clients:

Appropiate metric:

Minimize false creditworthy (= minimize possible losses)

Satisfying metric(s):

- High probabily of paying off. 

### Strategy to reach objective 2 

Solve the inverse problem, given a client data, what is the maximum credit ammount that 
makes a client still trustworthy. 


### Strategy to reach objective 3 
Estimate losses per year (regression?) per lended-ammount. Based on this, we can fix a interest rate per year per lend ammount. With this distrution we hopefully can estimate a variance. 

### Future improvements

All these models are based on the data provided future generalisations should take into account:

- External factors (inflation and other risk)

- Fluctuations, this could be address by having a large test set to experiment with. 

## Data sets

A. users.csv codebook:

- id: User’s unique identifier.
- monthly_income: User’s monthly declared income.
- monthly_outcome: User’s monthly declared outcome.
- class: Boolean value 1 if the client was good or 0 if bad.

B. credit_reports.csv codebook:

- user_id: User’s unique identifier. 
- institution: Institution granting the loan.
- account_type: Type of account for the institution.
- credit_type: Type of loan granted by the institution.   

- total_credit_payments: Length of the credit (in amount of payments).    
- payment_frequency: Frequency of the payments. 

- amount_to_pay_next_payment: Amount to be paid on the next loan payment.

- account_opening_date: Date the account was opened.
- account_closing_date: Date the account was closed.

- maximum_credit_amount: maximum amount of credit used by the consumer.
- current_balance: Current balance needed to pay off the loan.
- credit_limit: Credit limit for this account.
- past_due_balance: Balance that is delinquent.
- number_of_payments_due: Number of payments that are delinquent.
- worst_delinquency: Worst delinquency (in payments) during the loan’s life.
- worst_delinquency_date: Worst delinquency date.
- worst_delinquency_past_due_balance: Worst accumulated delinquent balance.

In [2]:
df_credit_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16309 entries, 0 to 16308
Data columns (total 17 columns):
user_id                               16309 non-null int64
institution                           16309 non-null object
account_type                          16309 non-null object
credit_type                           16309 non-null object
total_credit_payments                 16268 non-null float64
payment_frequency                     16308 non-null object
amount_to_pay_next_payment            16304 non-null float64
account_opening_date                  16309 non-null object
account_closing_date                  9068 non-null object
maximum_credit_amount                 16307 non-null float64
current_balance                       16304 non-null float64
credit_limit                          16296 non-null float64
past_due_balance                      16304 non-null float64
number_of_payments_due                16297 non-null float64
worst_delinquency                     16255 no

In [3]:
df_credit_reports.head(50)

,user_id,institution,account_type,credit_type,total_credit_payments,payment_frequency,amount_to_pay_next_payment,account_opening_date,account_closing_date,maximum_credit_amount,current_balance,credit_limit,past_due_balance,number_of_payments_due,worst_delinquency,worst_delinquency_date,worst_delinquency_past_due_balance
0,0,MERCANCIA PARA HOGAR Y OFICINA,Pagos Fijos,Línea de Crédito,78.0,Semanal,0.0,11/10/14,12/5/15,9826.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1,0,MERCANCIA PARA HOGAR Y OFICINA,Pagos Fijos,Préstamo Personal,80.0,Semanal,0.0,12/5/15,10/14/16,16000.0,0.0,0.0,0.0,3.0,1.0,5/28/16,200.0
2,0,MERCANCIA PARA HOGAR Y OFICINA,Pagos Fijos,Línea de Crédito,102.0,Semanal,100.0,9/2/16,NaN,10205.0,3505.0,0.0,0.0,3.0,1.0,3/3/17,100.0
3,0,MERCANCIA PARA HOGAR Y OFICINA,Pagos Fijos,Préstamo Personal,120.0,Semanal,191.0,10/14/16,NaN,22920.0,11269.0,0.0,0.0,3.0,1.0,3/3/17,191.0
4,0,KONFIO,Pagos Fijos,Préstamo Personal,12.0,Quincenal,0.0,12/27/16,6/27/17,150000.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
5,0,KONFIO,Pagos Fijos,Préstamo Personal,26.0,Quincenal,10825.0,6/28/17,NaN,225000.0,159142.0,0.0,0.0,0.0,0.0,NaN,0.0
6,0,TELEFONIA CELULAR,Revolvente,Línea de Crédito,1000.0,Mensual,0.0,10/6/16,9/15/17,0.0,0.0,999.0,0.0,7.0,4.0,5/31/17,4342.0
7,0,MERCANCIA PARA HOGAR Y OFICINA,Sin Límite Preestablecido,Crédito Personal al Consumo,0.0,Semanal,0.0,11/10/14,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
8,0,AUTOMOTRIZ,Pagos Fijos,Compra de Automóvil,72.0,Mensual,5874.0,8/22/17,NaN,513069.0,475245.0,513069.0,0.0,0.0,0.0,NaN,0.0
9,0,AUTOMOTRIZ,Pagos Fijos,Compra de Automóvil,24.0,Mensual,10557.0,1/31/17,NaN,247705.0,124316.0,247705.0,0.0,0.0,0.0,NaN,0.0


In [4]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
id                 1000 non-null int64
monthly_income     1000 non-null int64
monthly_outcome    1000 non-null int64
class              1000 non-null int64
dtypes: int64(4)
memory usage: 31.3 KB


In [5]:
df_users.head()

,id,monthly_income,monthly_outcome,class
0,0,255359,151439,1
1,1,8940,789,0
2,2,11436,3316,0
3,3,17143,47547,1
4,4,26990,15585,1
